## Option Pricing

In [1]:
from QuantumMC.quantummc import QuantumMC
from QuantumMC.variable import Variable
import numpy as np

In [2]:
from qiskit.circuit.library import LinearAmplitudeFunction
# number of qubits to represent the uncertainty
num_uncertainty_qubits = 3

# parameters for considered random distribution
S = 2.0  # initial spot price
vol = 0.4  # volatility of 40%
r = 0.05  # annual interest rate of 4%
T = 40 / 365  # 40 days to maturity

mu = (r - 0.5 * vol**2) * T + np.log(S)
sigma = vol * np.sqrt(T)
mean = np.exp(mu + sigma**2 / 2)
variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
stddev = np.sqrt(variance)

# lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
low = np.maximum(0, mean - 3 * stddev)
high = mean + 3 * stddev

# set the strike price (should be within the low and the high value of the uncertainty)
strike_price = 1.896

# set the approximation scaling for the payoff function
c_approx = 0.25

# setup piecewise linear objective fcuntion
breakpoints = [low, strike_price]
slopes = [0, 1]
offsets = [0, 0]
f_min = 0
f_max = high - strike_price
european_call_objective = LinearAmplitudeFunction(
    num_uncertainty_qubits,
    slopes,
    offsets,
    domain=(low, high),
    image=(f_min, f_max),
    breakpoints=breakpoints,
    rescaling_factor=c_approx,
)

In [3]:
qmc = QuantumMC() #Initialization
final_price = Variable(num_uncertainty_qubits)
final_price.load_distribution("LogNormal", num_uncertainty_qubits, mu=mu, sigma=sigma**2, bounds = (low, high))
qmc.add_variable(final_price)
result = qmc.estimate(0.05, 0.01, final_price, objective = european_call_objective)
result

0.18231197087131448

## Portfolio Selection

In [19]:
from QuantumMC.quantummc import QuantumMC
from QuantumMC.variable import Variable
import numpy as np

In [20]:
qmc = QuantumMC()
var1 = Variable(2, "var1")
var1.load_constant(3)
var2 = Variable(2, "var2")
var2.load_constant(2)
qmc.add_variable(var1)
qmc.add_variable(var2)

In [21]:
a, b = qmc.arithmetic(operation = "power2", variables = [var1])

In [22]:
from qiskit import *
cr = ClassicalRegister(b.get_register().size)
qmc.get_qc().add_register(cr)
for i in range(cr.size):
    qmc.get_qc().measure(b.get_register()[i], cr[i])
    
num_shots = 1000 #Setting the number of times to repeat measurement
backend = Aer.get_backend("qasm_simulator")
job = execute(qmc.get_qc(), backend, shots=num_shots)
#Get results of program
job_stats = job.result().get_counts()
print(job_stats)

a = 0
for i in job_stats:
    a +=(int(i,2) * job_stats[i])   
#     print(int(i,2))
    
a/num_shots

{'1000': 1000}


8.0

## Scenario 1

In [16]:
from QuantumMC.quantummc import QuantumMC
from QuantumMC.variable import Variable
import numpy as np

In [17]:
qmc = QuantumMC()
qw1 = qmc.walk(num_steps = 1, distribution = "Normal", size = 1, name = "r", mu = 1, sigma = 0.5)
qw2 = qmc.walk(num_steps = 2, distribution = "Normal", size = 1, name = "xi", mu = 1, sigma = 0.5)

In [18]:
a, b= qmc.arithmetic("add", [qw1.vars[-1], qw2.vars[-1]])
var = Variable(2, "constant")
var.load_constant(2)
qmc.add_variable(var)
a, b= qmc.arithmetic("add", [var, b])
a, b = qmc.arithmetic("power2", [b])

In [19]:
result = qmc.estimate(0.05, 0.01, b, plain = False)
result
#55

33.59241262557278

In [20]:
# result *  (2 ** b.num_qubits - 1)

In [21]:
from qiskit import *
cr = ClassicalRegister(b.get_register().size)
qmc.get_qc().add_register(cr)
for i in range(cr.size):
    qmc.get_qc().measure(b.get_register()[i], cr[i])
    
num_shots = 100 #Setting the number of times to repeat measurement
backend = Aer.get_backend("qasm_simulator")
job = execute(qmc.get_qc(), backend, shots=num_shots)
#Get results of program
job_stats = job.result().get_counts()
print(job_stats)

a = 0
for i in job_stats:
    a +=(int(i,2) * job_stats[i])   
#     print(int(i,2))
    
a/num_shots

{'00100000': 99, '00010000': 1}


31.84

## Scenario 2

In [22]:
from QuantumMC.quantummc import QuantumMC
from QuantumMC.variable import Variable
import numpy as np

In [23]:
qmc = QuantumMC()
qw1 = qmc.walk(num_steps = 1, distribution = "Normal", size = 1, name = "r")
qw2 = qmc.walk(num_steps = 1, distribution = "Normal", size = 1, name = "xi")
qmc.get_qc().barrier()

In [24]:
a, b= qmc.arithmetic("add", [qw1.vars[-1], qw2.vars[-1]])

var = Variable(1, "constant")
var.load_constant(1)
qmc.add_variable(var)

a, b= qmc.arithmetic("add", [var, b], pad = False)
a, b = qmc.arithmetic("power2", [b])

In [25]:
a, b, c= qmc.arithmetic("mult", [qw1.vars[0], b])

In [26]:
result = qmc.estimate(0.01, 0.01, c, plain = False)
result
#1.1

2.7283594879938358

In [27]:
from qiskit import *
cr = ClassicalRegister(c.get_register().size)
qmc.get_qc().add_register(cr)
for i in range(cr.size):
    qmc.get_qc().measure(c.get_register()[i], cr[i])

In [28]:
num_shots = 100 #Setting the number of times to repeat measurement
backend = Aer.get_backend("qasm_simulator")
job = execute(qmc.get_qc(), backend, shots=num_shots)
#Get results of program
job_stats = job.result().get_counts()
print(job_stats)

{'001000': 27, '000000': 55, '000100': 18}


In [29]:
a = 0
for i in job_stats:
    a +=(int(i,2) * job_stats[i])   
#     print(int(i,2))

In [30]:
a/num_shots

2.88